In [1]:
import json
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pandas as pd
import os

with open('../Secret_file.json',"r") as f:
    ks_token = json.load(f)['Slack_API']['KS']

In [ ]:
def Bot_Message(channel, text):
    payload={
        "token":ks_token,
        "channel":channel,
        "text":'{}'.format(text)} 
    r = requests.post("https://slack.com/api/chat.postMessage", params=payload) 

In [ ]:
yesterday = datetime.now().day

check_point = 0
while check_point <6:
    try:
        start_time = time.time()
        today = datetime.now()
        
        #token_page
        r = requests.get('https://www.kickstarter.com/discover/advanced?state=upcoming&category_id=34&woe_id=1118370')
        soup = BeautifulSoup(r.text, 'html5lib')
        token = soup.find('meta', attrs={'name':'csrf-token'})['content']
        cookies = r.cookies['_ksr_session']

        #detail_page
        url = 'https://www.kickstarter.com/graph'
        payload = {"operationName":"PrelaunchPage",
                   "variables":{"slug":"moaideas/steel-n-steam-hexagonal-metal-coins-for-train-games"},
                   "query":"query PrelaunchPage($slug: String!) {\n  me {\n    id\n    isKsrAdmin\n    __typename\n  }\n  project(slug: $slug) {\n    id\n    name\n    location {\n      discoverUrl\n      displayableName\n      countryName\n      __typename\n    }\n    state\n    description\n    url\n    imageUrl(width: 1000)\n    prelaunchActivated\n    isWatched\n    watchesCount\n    category {\n      url\n      name\n      parentCategory {\n        name\n        __typename\n      }\n      __typename\n    }\n    verifiedIdentity\n    creator {\n      id\n      name\n      slug\n      hasImage\n      imageUrl(width: 48)\n      url\n      biography\n      backingsCount\n      isFacebookConnected\n      lastLogin\n      websites {\n        url\n        domain\n        __typename\n      }\n      launchedProjects {\n        totalCount\n        __typename\n      }\n      location {\n        displayableName\n        __typename\n      }\n      __typename\n    }\n    collaborators {\n      edges {\n        node {\n          name\n          url\n          imageUrl(width: 48)\n          __typename\n        }\n        title\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n"}
        headers = {'content-type': 'application/json',
                   'cookie':'_ksr_session={}'.format(cookies),
                   'x-csrf-token': '{}'.format(token)}
        rp = requests.post(url, data=json.dumps(payload), headers=headers)
        count = rp.json()['data']['project']['watchesCount']
        df = pd.DataFrame([(today.strftime('%Y/%m/%d %H:%M'),count)], columns=['time', 'follow_count'])
        
        if os.path.exists('../../save_data/KS/老外金屬幣紀錄.csv'):
            data = pd.read_csv('../../save_data/KS/老外金屬幣紀錄.csv')
            data = data.append(df)
            data.to_csv('../../save_data/KS/老外金屬幣紀錄.csv', index=False)
            del data
        else:
            df.to_csv('../../save_data/KS/老外金屬幣紀錄.csv', index=False)
        
        if today.weekday() != 0:
            pass
        elif today.hour != 9:
            pass
        else:
            Bot_Message('data_analysis', '{}\n火車幣目前Follow人數：{:,}'.format(today.strftime('%Y/%m/%d'), count))
        
        time.sleep(3600-(time.time() - start_time))
    except:
        check_point += 1
        Bot_Message('spider_status', '【警告】\nKS_老外金屬幣專案發生錯誤({}/{})。'.format(check_point, 6))
        time.sleep(5)